In [ ]:
%matplotlib inline
import numpy as np
import torch
from torch import nn, optim
from torchvision import transforms, datasets
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

## Загрузка данных

In [ ]:
dataset  = datasets.MNIST(root='./data/',
                            train=True,
                            transform=transforms.Compose(
                                  [transforms.ToTensor(),
                                  ]),
                            download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)
num_batches = len(data_loader)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Метод для генерации рандомного шума (вход в генератор)

In [ ]:
def random_noise(batch_size):
    n = torch.randn(batch_size, 100)
    return n.to(device)

Метод для показа изображений (много преобразовательной магии, не вникайте в этот метод пожалуйста)

In [ ]:
def visualise(imgs, rows=2):
    imgs = imgs.view(-1, 1, 28, 28)
    imgs = (imgs.transpose(1, 3) + 1) / 2
    imgs = torch.cat([imgs[i::rows] for i in range(rows)], dim=1)
    cols = len(imgs)
    imgs = np.squeeze((torch.cat(list(imgs), dim=1)).cpu().numpy()[:, :, ::-1])
    plt.figure(figsize=(cols*1.5, rows*1.5))
    plt.imshow(imgs, cmap = 'gray')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## Модули

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        n_features = 784
        n_out = 1
        # вставить слои


    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

In [ ]:
class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        n_features = 100
        n_out = 784
        # вставить слои
        


    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x


In [ ]:
discriminator = Discriminator()
generator = Generator()
discriminator.to(device)
generator.to(device)

Generator(
  (hidden0): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden1): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (hidden2): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (out): Sequential(
    (0): Linear(in_features=1024, out_features=784, bias=True)
    (1): Tanh()
  )
)

## Оптимизация и функция потерь

In [ ]:
# Оптимизаторы
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

# Функция потерь
loss = nn.BCELoss()

num_epochs = 200

## Методы для обчуения генератора и дискриминатора

In [ ]:
def train_discriminator(optimizer, real_data, fake_data):
    # обнуляем градиенты

    # Обучение на настоящей выборке

    # Считаем лосс для настоящих данных и делаем backpropagate

    # Обучение на сгенерированной выборке

    # Считаем лосс для фейковых данных и делаем backpropagate

    
    # возвращаем ошибку а также выход дискриминатора на истинной картинке и фейковой
    return error_real + error_fake

def train_generator(optimizer, fake_data):
    # обнуляем градиенты

    # считаем лосс между сгенеренной картинкой и истинной

    return error

In [ ]:
visualise_every = 1

### Начинаем обучать

In [ ]:
for epoch in range(num_epochs):
    total_batches = 0
    gen_loss_avg = 0
    disc_loss_avg = 0
    for num, (real_batch, _) in tqdm(enumerate(data_loader), total=num_batches):

        total_batches += 1
        real_batch_size = real_batch.size(0)
        # Дискриминатор
        real_data = real_batch.view(real_batch_size, 784) # решейпаем в вектор
        real_data = real_data.to(device)
        # генерируем фейковые данные
        fake_data = generator(random_noise(real_batch_size))
        # обучаем дискриминатор
        d_loss = train_discriminator(d_optimizer,
                                    real_data, fake_data)
        disc_loss_avg += d_loss.item()

        # Генератор
        # генерируем фейковые данные
        fake_data = generator(random_noise(real_batch_size))
        # обучаем генератор
        g_loss = train_generator(g_optimizer, fake_data)
        gen_loss_avg += g_loss.item()

        # показать прогресс
    if (epoch + 1) % visualise_every == 0:
        with torch.no_grad():
            visualise(generator(random_noise(6)), rows=2)
    print(f"Epoch {epoch+1} | Discriminator loss: {disc_loss_avg / total_batches} | Generator loss: {gen_loss_avg / total_batches}")


  0%|          | 0/600 [00:00<?, ?it/s]

Epoch 1 | Discriminator loss: 1.3306108854214351 | Generator loss: 1.1208074996620416


  0%|          | 0/600 [00:00<?, ?it/s]

Epoch 2 | Discriminator loss: 1.0383010350664457 | Generator loss: 2.021383197704951


  0%|          | 0/600 [00:00<?, ?it/s]